In [1]:
# default_exp core

# Core

> API details.

In [2]:
#export
from dataclasses import dataclass
from pathlib import Path
import datetime, json

In [3]:
#export
def now(): 
    return datetime.datetime.utcnow().strftime('%Y%m%d_%H%M%S')

In [4]:
#export
def get_dict_subset(d, name_prefix, remove_prefix=True, remove_additional_characters=1):
    "Return a subset of entries from a dictionary like object"
    if name_prefix is None or name_prefix == '': return d
    def _rm_prefix(k):
        if not remove_prefix: return k
        return k[len(name_prefix)+remove_additional_characters:]
    return {_rm_prefix(k): d[k] for k in d if k.startswith(name_prefix)}

A pytorch state dict will have keys like
```
[...
 'layers.0.0.weight',
 'layers.0.2.weight',
 'layers.0.2.bias',
 'layers.0.2.running_mean',
 'layers.0.2.running_var',
 'layers.0.2.num_batches_tracked',
 ...
 'layers.2.0.weight',
 'layers.2.0.bias']
```
so can get all state for a module with something like;
```
get_dict_subset(state_dict, 'layers.2.0')
```
which returns
```
{'weight': [...], 'bias': [...]}
```

In [5]:
test_dict = {}
for p1 in ['a','b','c']:
    for p2 in ['a','b','c']:
        test_dict[f'{p1}.{p2}']=f'{p1}.{p2}.value'

expected = {'a': 'b.a.value', 'b': 'b.b.value', 'c': 'b.c.value'}
assert get_dict_subset(test_dict, 'b') == expected
expected = {'b.a': 'b.a.value', 'b.b': 'b.b.value', 'b.c': 'b.c.value'}
assert get_dict_subset(test_dict, 'b', False) == expected
expected = {'.a': 'b.a.value', '.b': 'b.b.value', '.c': 'b.c.value'}
assert get_dict_subset(test_dict, 'b', True, 0) == expected

In [6]:
#export
@dataclass(frozen=True) # frozen doesn't really help (o: but we don't want to change values of this data class
class FaceLandmarks:
    count=468
    top_lip_indent=0
    tip_of_nose=1
    pointer = [1, 5, 2, 218, 438] # tip_of_nose,up,down,left,right
    mouth = [0, 11, 12, 13, 14, 15, 16, 17, 18, 37, 38, 39, 40, 41, 42, 43, 57, 61, 62, 72, 73, 74, 76, 77, 78, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 95, 96, 106, 146, 164, 165, 167, 178, 179, 180, 181, 182, 183, 184, 185, 186, 191, 204, 267, 268, 269, 270, 271, 272, 273, 287, 291, 292, 302, 303, 304, 306, 307, 308, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 324, 325, 335, 375, 391, 393, 402, 403, 404, 405, 406, 407, 408, 409, 410, 415, 424]

In [15]:
#export
class VisemeConfig:
    def __init__(self, path='data'):
        with open(Path(path)/'viseme-config.json') as f: 
            self.config = json.load(f)
    def get_class_label(self, class_id):
        return self.config['expressions'][str(class_id)]
    def get_class_ids_and_labels(self):
        items = self.config['expressions'].items()
        return [int(i) for i, j in items], [j for i, j in items]
        # we could use the bellow, if we didn't have to convert class IDs to int
        # return list(zip(*self.config['expressions'].items()))

In [17]:
viseme_config = VisemeConfig()
assert 'oo' == viseme_config.get_class_label(1)
class_ids, class_labels = viseme_config.get_class_ids_and_labels()
assert class_ids == [0, 1, 2, 3, 4]
assert class_labels == ['No expression', 'oo', 'ee', 'ah', 'Random Talking']

In [9]:
#export
class LogFile:
    def __init__(self, path):
        self.path = Path(path)
        print('LogFile:', self.path.resolve())
    def __call__(self, *args):
        with open(self.path, 'a') as f:
            f.write(' '.join([str(arg) for arg in args]) + '\n')
        return self

In [10]:
import tempfile, shutil

temp_path = Path(tempfile.mkdtemp())
try:
    log = LogFile(temp_path/'a.log')
    log('line1')
    log(2)
    log('line3\nline4')
    log('line5')(6) # TODO: chaining is possible but I'm not sure it's a good idea?
    (log('line7')
     ('line8')
     (9))
    expected = "line1.2.line3.line4.line5.6.line7.line8.9.".replace('.', '\n')
    with open(temp_path/'a.log') as f:
        actual = f.read()
    assert actual == expected
finally:
    shutil.rmtree(temp_path)

LogFile: C:\Users\Butterp\AppData\Local\Temp\tmpocae2odd\a.log


In [18]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 05a_ml_data.ipynb.
Converted 05b_ml_model.ipynb.
Converted 10a_mediapipe_face_mesh_identify_landmarks.ipynb.
No export destination, ignored:
#export
import ipywidgets as widgets
import numpy as np
import pandas as pd
import cv2, time, math, json, shutil
import win32api, win32con

import mediapipe as mp
mp_face_mesh = mp.solutions.face_mesh

from pathlib import Path
No export destination, ignored:
#export
def _now(): 
    return datetime.datetime.utcnow().strftime('%Y%m%d_%H%M%S')
No export destination, ignored:
#export
def _new_capture_metadata(stop_after, path, video_capture, expression_id, comments):
    width, height = [int(video_capture.get(p)) for p in [cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT]]
    assert width >= height
    return dict(count=0, stop_after=stop_after, path=str(path.resolve()), expression_id=expression_id,
                capture_width=width, capture_height=height, start_date=_now(), comments=comments)
No export destin

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Butterp\\github\\pete88b\\expoco\\expoco\\None.py'